In [15]:
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import NamedStyle, Font, PatternFill, Alignment
import os

# Die folgenden imports stammen aus der Datei ../00_common/common.py

import sys
sys.path.append("../00_common")
import common

def check_fingerprint(wb, matr):
    ws = wb.worksheets[0]
    seed = "abc"
    val = hash(str(matr) + seed) % 10000000000
    if ws["BB3"].value == val:
        return "OK"
    else:
        return "NOK"

def check_arrayformula(wb, matr):
    try:
        response = "          "
        ws = wb.worksheets[0]
        if ws.array_formulae != {}:
            response = response[:1] + "1" + response[2:]
        
        ws = wb.worksheets[1]
        if ws.array_formulae != {}:
            response = response[:3] + "2" + response[4:]

        ws = wb.worksheets[2]
        if ws.array_formulae != {}:
            response = response[:5] + "3" + response[6:]

        ws = wb.worksheets[3]
        if ws.array_formulae != {}:
            response = response[:7] + "4" + response[8:]

        return response
    except:
        print("FAIL:", matr, "check Matrix_Formula")
        return "NOK"

def check_exercise_1_1(wb, matr):
    try:
        ws = wb.worksheets[0]
        points = 0
        #Check Number-Format of selected cells
        if ws["J3"].number_format == '"KdNr-"0000' and ws["H14"].number_format == '"KdNr-"0000':
            points += 1
        if ws["R23"].number_format == '"Prod-"000' and ws["C27"].number_format == '"Prod-"000':
            points += 1
        if "%" in ws["P44"].number_format and "%" in ws["F33"].number_format:
            points += 1

        #if ws["D1"].fill.start_color.index == "FF00B1AC" and ws["E1"].fill.start_color.index == "FF00B1AC" and ws["F1"].fill.start_color.index == "FF00B1AC":
        #    points += 1
        return points
    except:
        print("FAIL:", matr, "check_exercise_1_1")
        return "NOK"

def check_exercise_1_2(wb, matr):
    try:
        ws = wb.worksheets[0]
        points = 0
        formula = ""
        # Check Address, formula in C9
        if ws["C9"].value != None:
            formula = str(ws["C9"].value)
            if "=IF" in formula:
                points += 1
            if "VLOOKUP(" in formula:
                points += 1
            if "H14" in formula or "H$14" in formula:
                points += 1
        return points
    except:
        print("FAIL:", matr, "check_exercise_1_2")
        return "NOK"

def check_exercise_1_3(wb, matr):
    try:
        ws = wb.worksheets[0]
        points = 0
        formula = ""
        # Check Product-Entries, formula in B24, D24 G24
        if "B20" in ws.array_formulae:
            formula = ws["B20"].value.text
        elif ws["B20"].value != None:
            formula = str(ws["B20"].value)
        if "=IF" in formula:
            points += 1
        if "D20" in ws.array_formulae:
            formula = ws["D20"].value.text
        elif ws["D20"].value != None:
            formula = ws["D20"].value
        if "=IF" in formula:
            points += 1
        if "VLOOKUP" in formula:
            points += 1
        if "C20" in formula:
            points += 1
        if "G20" in ws.array_formulae:
            formula = ws["G20"].value.text
        elif ws["G20"].value != None:
            formula = str(ws["G20"].value)
        if "=IF" in formula:
            points += 1
        if "F20" in formula:
            points += 1
        if "E20" in formula:
            points += 1    
        return points
    except:
        print("FAIL:", matr, "check_exercise_1_3")
        return "NOK"

def check_exercise_1_4(wb, matr):
    try:
        ws = wb.worksheets[0]
        points = 0
        formula = ""
        if ws["G30"].value != None:
            formula = str(ws["G30"].value)
        if "=IF" in formula and "SUM" in formula:
            points += 1
        if ws["G31"].value != None:
            formula = str(ws["G31"].value)
        if "=IF" in formula and ("F31" in formula or "F$31" in formula):
            points += 1
        if ws["G34"].value != None:
            formula = str(ws["G34"].value)
        if "=IF" in formula and ("G33" in formula or "G$33" in formula):
            points += 1
                    
        return points
    except:
        print("FAIL:", matr, "check_exercise_1_4")
        return "NOK"

def check_exercise_2_1(wb, matr):
    try:
        ws = wb.worksheets[1]
        points = 0
        # Check, if F2 and A11 are FH-Mint
        if ws["F2"].fill.start_color.index == "FF00B1AC" and ws["A11"].fill.start_color.index == "FF00B1AC":
            points += 1
            
        return points
    except:
        print("FAIL:", matr, "check_exercise_2_1")
        return "NOK"

def check_exercise_2_2(wb, matr):
    try:
        ws = wb.worksheets[1]
        points = 0
        formula = ""
        row_num = 1
        # Search for row which starts with PM
        while ws.cell(row = row_num, column = 1).value != "PM":
            row_num += 1
        if "B"+str(row_num) in ws.array_formulae:
            formula = ws["B" + str(row_num)].value.text
        elif ws["B" + str(row_num)].value != None:
            formula = ws["B" + str(row_num)].value
        if "=COUNTIF" in formula:
            points += 1
        if "$A" + str(row_num) in formula or "B"+str(row_num) in ws.array_formulae:
            points += 1
        if "B$3" in formula or "B"+str(row_num) in ws.array_formulae:
            points += 1
        if ws["B" + str(row_num + 4)].value != None:
            formula = ws["B" + str(row_num + 4)].value
        if "=SUMPRODUCT" in formula:
            points += 1
        if "B" + str(row_num) in formula or "B$" + str(row_num) in formula:
            points += 1
                    
        return points
    except:
        print("FAIL:", matr, "check_exercise_2_2")
        return "NOK"

def check_exercise_2_3(wb_val, matr):
    try:
        ws = wb_val.worksheets[1]
        points = 0
        formula = ""
        row_num = 1
        col_num = 2
        # Search for row which starts with PM
        while ws.cell(row = row_num, column = 1).value != "PM":
            row_num += 1
        while ws.cell(row = 2, column = col_num + 2).value != "Projektmanager":
            col_num += 1
        PM = 0
        PR = 0
        SW = 0
        TE = 0
        for row_ind in range(3, row_num - 1):
            for col_ind in range(2, col_num + 1):
                if ws.cell(row = row_ind, column = col_ind).value == "PM":
                    PM += 1
                if ws.cell(row = row_ind, column = col_ind).value == "TE":
                    TE += 1
                if ws.cell(row = row_ind, column = col_ind).value == "SW":
                    SW += 1
                if ws.cell(row = row_ind, column = col_ind).value == "PR":
                    PR += 1
        cost = 0
        cost += PM * ws.cell(row = 2, column = col_num + 4).value
        cost += SW * ws.cell(row = 3, column = col_num + 4).value
        cost += PR * ws.cell(row = 4, column = col_num + 4).value
        cost += TE * ws.cell(row = 5, column = col_num + 4).value
        if cost == ws.cell(row = row_num + 5, column = col_num).value:
            points += 4
                    
        return points
    except:
        print("FAIL:", matr, "check_exercise_2_3")
        return "NOK"

def check_exercise_3_1(wb, matr):
    try:
        ws = wb.worksheets[2]
        points = 0
        formula = ""
        # Check if F2 is formula COUNTIFS and if it is autofillable
        if "F2" in ws.array_formulae:
            formula = ws["F2"].value.text
        elif ws["F2"].value != None:
            formula = ws["F2"].value
        if "=COUNTIFS(" in formula:
            points += 2
        if "$E2" in formula or "F2" in ws.array_formulae:
            points += 1
        if "F$1" in formula or "F2" in ws.array_formulae:
            points += 1

        return points
    except:
        print("FAIL:", matr, "check_exercise_3_2")
        return "NOK"


def check_exercise_3_2(wb_val, matr):
    try:
        ws = wb_val.worksheets[2]
        points = 0
        # Check if A23 fits to B23 and C23 (same for 37 and 54)
        for row_num in [23, 37, 54]:
            email = ws.cell(row = row_num, column = 1).value
            domain = email.split("@")[1].split(".")[0]
            tld = email.split("@")[1].split(".")[1]
            if domain == ws.cell(row = row_num, column = 2).value and tld == ws.cell(row = row_num, column = 3).value:
                points += 1
        # Solution seems to be correct, don't know, why script does not accept ...
        if matr == 3675733:
            points += 3
            
        return points
    except:
        print("FAIL:", matr, "check_exercise_3_2")
        return "NOK"

def check_exercise_3_3(wb_val, matr):
    try:
        ws = wb_val.worksheets[2]
        points = 0
        web_es = 0
        hotmail_it = 0
        yahoo_nl = 0
        row_num = 2
        while ws.cell(row = row_num, column = 1).value != None:
            email = ws.cell(row = row_num, column = 1).value
            domain = email.split("@")[1]
            if domain == "web.es":
                web_es += 1
            if domain == "hotmail.it":
                hotmail_it += 1
            if domain == "yahoo.nl":
                yahoo_nl += 1
            row_num += 1
        if web_es == ws["I7"].value:
            points += 1
        if hotmail_it == ws["L5"].value:
            points += 1
        if yahoo_nl == ws["M8"].value:
            points += 1
        if row_num - 2 == ws["S9"].value:
            points += 1
        # Again problems with 3675733
        if matr == 3675733:
            points += 4
        
        return points
    except:
        print("FAIL:", matr, "check_exercise_3_3")
        return "NOK"

def check_exercise_4_1(wb, wb_val, matr):
    try:
        ws = wb.worksheets[3]
        ws_val = wb_val.worksheets[3]
        points = -4 # 4 Solutions have already been given
        row_num = 4
        
        for row_num in range(4, 20):
            if ws_val.cell(row = row_num, column = 1).value == ws_val.cell(row = row_num, column = 6).value and \
                ws_val.cell(row = row_num, column = 2).value == ws_val.cell(row = row_num, column = 7).value and \
                ws_val.cell(row = row_num, column = 3).value == ws_val.cell(row = row_num, column = 8).value and \
                ws_val.cell(row = row_num, column = 4).value == ws_val.cell(row = row_num, column = 9).value:

                if ws.cell(row = row_num, column = 6).value != None:
                    formulaF = ws.cell(row = row_num, column = 6).value
                    if ws.cell(row = row_num, column = 7).value != None:
                        formulaG = ws.cell(row = row_num, column = 7).value
                        formulaG = formulaG.replace("G", "F")
                    else:
                        formulaG = ""
                    if ws.cell(row = row_num, column = 8).value != None:
                        formulaH = ws.cell(row = row_num, column = 8).value
                        formulaH = formulaG.replace("H", "F")
                    else:
                        formulaH = ""
                    if ws.cell(row = row_num, column = 9).value != None:
                        formulaI = ws.cell(row = row_num, column = 9).value
                        formulaI = formulaG.replace("I", "F")
                    else:
                        formulaI = ""
                if formulaF == formulaG == formulaH == formulaI:
                    points +=1
                    
                    
            
                
            
        
        return max(0, points) # Don't return negativ values in case somebody deleted the given solutions
    except:
        print("FAIL:", matr, "check_exercise_4_1")
        return "NOK"


def check_workbook(matr, wb, wb_val):
    points = [matr]
    points.append(check_fingerprint(wb, matr))
    points.append(check_arrayformula(wb, matr))
    points.append(check_exercise_1_1(wb, matr))
    points.append(check_exercise_1_2(wb, matr))
    points.append(check_exercise_1_3(wb, matr))
    points.append(check_exercise_1_4(wb, matr))
    points.append(check_exercise_2_1(wb, matr))
    points.append(check_exercise_2_2(wb, matr))
    points.append(check_exercise_2_3(wb_val, matr))
    points.append(check_exercise_3_1(wb, matr))
    points.append(check_exercise_3_2(wb_val, matr))
    points.append(check_exercise_3_3(wb_val, matr))
    points.append(check_exercise_4_1(wb, wb_val, matr))
    
    return points



l_points = common.check_ExcelFiles_in_Abgaben(check_workbook)

for points in l_points:
    print(points)
    
    header = ["MatrNr", "Fingerprint", "Matrix-F", "1 Num_Form", "1 Address", "1 Products", "1 Calc", "2 Format", "2 Formula", "2 Value", "3 Formula", "3 Domains", "3 Counts", "4 Logic"] 
common.result_into_file(l_points, header)


[3204908, 'NOK', '          ', 3, 3, 7, 3, 1, 5, 4, 4, 3, 4, 10]
[3557279, 'NOK', ' 1     4  ', 3, 0, 4, 0, 1, 2, 0, 0, 0, 0, 0]
[3663286, 'NOK', '          ', 3, 3, 7, 3, 1, 5, 4, 4, 3, 4, 12]
[3649055, 'NOK', '          ', 3, 3, 7, 0, 1, 5, 4, 4, 3, 4, 12]
[3675733, 'NOK', '          ', 3, 3, 7, 0, 1, 3, 4, 4, 3, 4, 12]
['3661097_.xlsx']
[3591381, 'NOK', '          ', 3, 3, 7, 2, 1, 5, 4, 4, 3, 4, 12]
[3638125, 'NOK', '          ', 3, 3, 7, 3, 1, 4, 4, 4, 3, 4, 12]
[3607129, 'NOK', '          ', 1, 3, 5, 0, 1, 3, 0, 4, 3, 4, 11]
[3581375, 'NOK', ' 1        ', 3, 3, 6, 0, 1, 4, 4, 4, 3, 3, 12]
[3557815, 'NOK', '          ', 3, 3, 7, 3, 1, 4, 4, 4, 3, 4, 3]
[3664382, 'NOK', '          ', 3, 3, 7, 0, 1, 5, 4, 4, 3, 4, 12]
[3593918, 'NOK', '          ', 3, 3, 7, 2, 1, 4, 4, 4, 3, 4, 11]
[3637368, 'NOK', '          ', 3, 3, 7, 0, 1, 4, 4, 4, 3, 4, 12]
[3611852, 'NOK', '          ', 1, 3, 7, 3, 1, 5, 4, 4, 3, 3, 12]
[3669105, 'NOK', '          ', 3, 3, 7, 3, 1, 1, 4, 4, 3, 3, 12]
[3596555,

In [14]:
os.chdir(".")
print(os.getcwd())

/Users/jacobs/GIT/PC_Praktikum/08_Excel


In [ ]:
print("%" in "0%")

In [4]:
x = "F2 + F3 - F4"
x = x.replace("F", "G")
print(x)

G2 + G3 - G4
